In [ ]:
import json
import os

In [ ]:
jsonfiles = ["/root/data/gtsf_phase_I/2019-02-26/2019-02-26_cogito_annotations.json",
             "/root/data/gtsf_phase_I/2019-02-27/2019-02-27_cogito_annotations.json",
             "/root/data/gtsf_phase_I/2019-03-01/2019-03-01_cogito_annotations.json",
             "/root/data/gtsf_phase_I/2019-03-04/2019-03-04_cogito_annotations.json",
             "/root/data/gtsf_phase_I/2019-03-05/2019-03-05_cogito_annotations.json",
             "/root/data/gtsf_phase_I/2019-03-06/2019-03-06_cogito_annotations.json",
             "/root/data/gtsf_phase_I/2019-03-11/2019-03-11_cogito_annotations.json",
             "/root/data/gtsf_phase_I/2019-03-13/2019-03-13_cogito_annotations.json",
             "/root/data/gtsf_phase_I/2019-03-14/2019-03-14_cogito_annotations.json"]

In [ ]:
annotations = []
for jsonpath in jsonfiles:
    with open(jsonpath, "r") as f:
        jfile = json.load(f)
        annotations += jfile
print("Number of annotations: {}".format(len(annotations)))

In [ ]:
for ann in annotations:
    local_path = os.path.join("/root/data/gtsf_phase_I/", 
                  "/".join(ann["Labeled Data"].split("/")[7:]))
    ann["local_path"] = local_path
    if not os.path.isfile(local_path):
        print("missing image!!")
    for body_part in ann["Label"].keys():
        new_body_part = "_".join(body_part.replace(":", "").split()).upper()
        ann["Label"][new_body_part] = ann["Label"].pop(body_part)

In [ ]:
timestamps = []
for ann in annotations:
    timestamp = ann["local_path"].split("/")[-3]
    ann["timestamp"] = timestamp
    timestamps.append(ann["timestamp"])

In [ ]:
# create pairs per timestamp
pairs = {}
for ann in annotations:
    timestamp = ann["timestamp"]
    side = os.path.basename(ann["local_path"]).split("_")[0]
    ann["side"] = side
    if timestamp not in pairs:
        pairs[timestamp] = {}
    pairs[timestamp][side] = ann

full_pairs = [k for (k, v)in pairs.items() if "left" in v and "right" in v]
print("Number of full pairs: {}".format(len(full_pairs)))

Select a random pair and write input file

In [ ]:
import numpy as np
from utils import load_keypoints

In [ ]:
input_json = {}
random_timestanp = np.random.choice(full_pairs)

In [ ]:
random_timestanp = '190306010013'

In [ ]:
mapping = {'ADIPOSE_FIN': 5,
 'ANAL_FIN': 1,
 'DORSAL_FIN': 4,
 'EYE': 7,
 'PECTORAL_FIN': 0,
 'PELVIC_FIN': 6,
 'TAIL_NOTCH': 3,
 'UPPER_LIP': 2}

In [ ]:
world = {}
for ts in full_pairs:
    # load annotations
    left_ann = pairs[ts]["left"]
    right_ann = pairs[ts]["right"]
    
    left_keypoints = load_keypoints(left_ann, mapping)
    right_keypoints = load_keypoints(right_ann, mapping)

In [ ]:
left_keypoints

In [ ]:
right_keypoints

In [ ]:
left_crop = []
right_crop = []
for (k,v) in mapping.items():
    tmp = {}
    tmp["xFrame"] = int(left_keypoints[v, 1])
    tmp["yFrame"] = int(left_keypoints[v, 0])
    tmp["xCrop"] = 0
    tmp["yCrop"] = 0
    tmp["keypointType"] = k
    left_crop.append(tmp)
    
    tmp = {}
    tmp["xFrame"] = int(right_keypoints[v, 1])
    tmp["yFrame"] = int(right_keypoints[v, 0])
    tmp["xCrop"] = 0
    tmp["yCrop"] = 0
    tmp["keypointType"] = k
    right_crop.append(tmp)

In [ ]:
left_crop

In [ ]:
right_crop

In [ ]:
cameraParameters = {
      "baseline": 0.128096,
      "focalLength": 0.0084366,
      "pixelCountWidth": 4096,
      "focalLengthPixel": 2445.3913043478265,
      "imageSensorWidth": 0.01412,
      "pixelCountHeight": 3000,
      "imageSensorHeight": 0.01035
  }

In [ ]:
input_json["leftCrop"] = left_crop
input_json["rightCrop"] = right_crop
input_json["cameraParameters"] = cameraParameters
input_json["annotationId"] = 10000
tmp["version"] = 1

In [ ]:
with open("./sample_message.json", "w") as f:
    json.dump(input_json, f)